then finish fine tuning using Lora
and do the classification again on all 3 datasets
oversample 2020 using smote


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

In [58]:
df1 = pd.read_json('/content/drive/MyDrive/Colab Notebooks/dissertation/data/2020_noFT.json')

df2 = pd.read_json('/content/drive/MyDrive/Colab Notebooks/dissertation/data/2019_noFT.json')

df = pd.concat([df1, df2], ignore_index=True)

# Create the StandardScaler object
scaler = StandardScaler()

In [61]:
df

,Chief Complaint,Predict,Consensus,emb_biogpt_no_FineT,emb_Stanford_no_FineT,emb_LLaMA_no_FineT
0,"Altercation while making arrest, c/o R hand pa...",N,N,"[[0.0141769592, 1.0703889132, -0.3458461463, -...","[[-1.9086047411, -0.82944417, 0.6341077089, -2...","[[-0.063993305, -0.7508783937, 0.077450037, -2..."
1,Cut on L upper thigh wtih saw. Bleeding contro...,N,N,"[[-0.25635513660000003, -1.5399489403, -0.2716...","[[-2.2433929443, -2.4271149635, 2.2515320778, ...","[[-1.142950654, -1.1670117378, 0.4413070977000..."
2,Generalized pain all over. Patient not answeri...,N,N,"[[-1.118257761, -0.5657621622, -2.4232051373, ...","[[2.2665572166, -2.2414543629, -0.8722355962, ...","[[-2.1129620075, -1.2800078392, -0.1476729363,..."
3,"R knee pain and swelling x2 weeks. ""I've just ...",U,N,"[[-0.3429946005, -0.1585950851, -1.94019198420...","[[-1.0249686241, 1.502045989, 0.89535653590000...","[[-1.5126222372, 0.2847052813, -0.306862145700..."
4,R wrist pain/swelling/numbness since this morn...,U,N,"[[-1.1331700087, 0.2827310562, -0.7864428163, ...","[[-2.1526670456, -0.10646272450000001, 0.78903...","[[-0.3649587333, -0.7817160487, 0.1761958003, ..."
...,...,...,...,...,...,...
558,"unwitnessed seizure last night, no dialysis in...",N,N,"[[0.7233701944, 0.0476501323, -2.4842305183, -...","[[1.5159145594, 1.7410211563, 0.5374487042, -1...","[[-0.8898552656000001, -0.0962213725, -0.02305..."
559,"upper abd/R side chest pain x1 month, new onse...",N,N,"[[-0.1761449426, 1.5961328745, 0.3407876492, -...","[[-1.7635282278000002, 2.4325175285, 1.2939627...","[[-0.4914414287, -0.6700435281, 0.2099424601, ..."
560,"upper lip swelling x one day, pmh HTN, COPD, b...",N,N,"[[0.6994464993, 1.261431694, -1.8226729631, -1...","[[-1.6149325371, 3.6044323444, -1.4300086498, ...","[[-1.3824286461, -0.5182033777, -0.7089595795,..."
562,was getting prepped for colonoscopy and was se...,N,N,"[[0.32566601040000004, -0.4120418131, -1.20206...","[[0.3229598403, 2.0347685814, 0.94253301620000...","[[-1.6664118767, 0.39107346530000003, 0.348639..."


###Classification on the Consensus column only 2 labels


In [60]:
# Check for consensus and mark rows for removal
rows_to_drop = []
for index, row in df.iterrows():
    if row['Consensus'] == '-' or row['Consensus'] == 'U': #or row['Consensus'] == 'U'
        rows_to_drop.append(index)

# Drop the marked rows
df = df.drop(rows_to_drop)

# Print the updated DataFrame
#print(df_2_labels)

In [62]:
X_biogpt, y_biogpt = df.emb_biogpt_no_FineT.tolist(), df.Consensus
X_stan, y_stan = df.emb_Stanford_no_FineT.tolist(), df.Consensus
X_llama, y_llama = df.emb_LLaMA_no_FineT.tolist(), df.Consensus

X_biogpt = np.squeeze(X_biogpt)
X_stan = np.squeeze(X_stan)
X_llama = np.squeeze(X_llama)

In [63]:
# Fit the scaler on the training data and transform it
X_biogpt = scaler.fit_transform(X_biogpt)
X_stan = scaler.fit_transform(X_stan)
X_llama = scaler.fit_transform(X_llama)

In [64]:
label_counts = y_biogpt.value_counts()

print(label_counts)

N    350
Y     95
Name: Consensus, dtype: int64


In [65]:
from sklearn.svm import SVC
#create SVC with class weight for predict column
svm_model_biogpt = SVC()
svm_model_stanford = SVC()
svm_model_llama = SVC()

In [66]:
def tunning(svc, X, y):
    param_grid = {'degree': [1, 2, 3],
                  'kernel': ['poly'],
                  'class_weight': ['balanced'],
                  'gamma':[0.01, 0.1, 1],
                  'C': [0.001, 0.1, 1]}

    grid = GridSearchCV(svc, param_grid, refit = True, verbose = 3, scoring= 'f1_macro', return_train_score=True, cv=5)

    # fitting the model for grid search
    grid.fit(X, y)

    # print best parameter after tuning
    best_params= grid.best_estimator_

    print(best_params)

    # print how our model looks after hyper-parameter tuning
    return best_params

In [67]:
#get best parameters for svc
svm_model_biogpt=tunning(svm_model_biogpt, X_biogpt, y_biogpt)
svm_model_stanford=tunning(svm_model_stanford, X_stan, y_stan)
svm_model_llama=tunning(svm_model_llama, X_llama, y_llama)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5] END C=0.001, class_weight=balanced, degree=1, gamma=0.01, kernel=poly;, score=(train=0.176, test=0.176) total time=   0.1s
[CV 2/5] END C=0.001, class_weight=balanced, degree=1, gamma=0.01, kernel=poly;, score=(train=0.176, test=0.176) total time=   0.1s
[CV 3/5] END C=0.001, class_weight=balanced, degree=1, gamma=0.01, kernel=poly;, score=(train=0.176, test=0.176) total time=   0.1s
[CV 4/5] END C=0.001, class_weight=balanced, degree=1, gamma=0.01, kernel=poly;, score=(train=0.176, test=0.176) total time=   0.1s
[CV 5/5] END C=0.001, class_weight=balanced, degree=1, gamma=0.01, kernel=poly;, score=(train=0.176, test=0.176) total time=   0.1s
[CV 1/5] END C=0.001, class_weight=balanced, degree=1, gamma=0.1, kernel=poly;, score=(train=0.663, test=0.487) total time=   0.1s
[CV 2/5] END C=0.001, class_weight=balanced, degree=1, gamma=0.1, kernel=poly;, score=(train=0.648, test=0.698) total time=   0.1s
[CV 3/5] END C=0

In [68]:
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

X_biogpt_train, X_biogpt_test, y_biogpt_train, y_biogpt_test = train_test_split(X_biogpt, y_biogpt, test_size=0.3, random_state=42)
X_stan_train, X_stan_test, y_stan_train, y_stan_test = train_test_split(X_stan, y_stan, test_size=0.3, random_state=42)
X_llama_train, X_llama_test, y_llama_train, y_llama_test = train_test_split(X_llama, y_llama, test_size=0.3, random_state=42)


# Instantiate SMOTE for each dataset
smote = SMOTE(random_state=42, sampling_strategy="minority", k_neighbors=5)

# Apply SMOTE to each training set
X_biogpt_oversampled, y_biogpt_oversampled = smote.fit_resample(X_biogpt_train, y_biogpt_train)
X_stan_oversampled, y_stan_oversampled = smote.fit_resample(X_stan_train, y_stan_train)
X_llama_oversampled, y_llama_oversampled = smote.fit_resample(X_llama_train, y_llama_train)


svm_model_biogpt.fit(X_biogpt_oversampled, y_biogpt_oversampled)
svm_model_stanford.fit(X_stan_oversampled, y_stan_oversampled)
svm_model_llama.fit(X_llama_oversampled, y_llama_oversampled)


# Predict on the original test sets
y_biogpt_pred = svm_model_biogpt.predict(X_biogpt_test)
y_stan_pred = svm_model_stanford.predict(X_stan_test)
y_llama_pred = svm_model_llama.predict(X_llama_test)

# Print classification report for each model
print("Classification Report for BioGPT SVM Model lett with oversampling:")
print(classification_report(y_biogpt_test, y_biogpt_pred))

print("Classification Report for Stanford SVM Model lett with oversampling:")
print(classification_report(y_stan_test, y_stan_pred))

print("Classification Report for LLAMA SVM Model lett with oversampling:")
print(classification_report(y_llama_test, y_llama_pred))

Classification Report for BioGPT SVM Model lett with oversampling:
              precision    recall  f1-score   support

           N       0.92      0.87      0.89       113
           Y       0.44      0.57      0.50        21

    accuracy                           0.82       134
   macro avg       0.68      0.72      0.70       134
weighted avg       0.84      0.82      0.83       134

Classification Report for Stanford SVM Model lett with oversampling:
              precision    recall  f1-score   support

           N       0.88      0.88      0.88       113
           Y       0.36      0.38      0.37        21

    accuracy                           0.80       134
   macro avg       0.62      0.63      0.63       134
weighted avg       0.80      0.80      0.80       134

Classification Report for LLAMA SVM Model lett with oversampling:
              precision    recall  f1-score   support

           N       0.88      0.85      0.86       113
           Y       0.32      0.38  

In [69]:
label_counts = y_biogpt_test.value_counts()

print(label_counts)

N    113
Y     21
Name: Consensus, dtype: int64


In [70]:
filtered_values = y_biogpt_test[y_biogpt_test == "Y"]
print(filtered_values)

513    Y
71     Y
80     Y
437    Y
361    Y
499    Y
73     Y
74     Y
394    Y
507    Y
244    Y
547    Y
400    Y
463    Y
502    Y
270    Y
532    Y
418    Y
495    Y
64     Y
29     Y
Name: Consensus, dtype: object


In [75]:
indices_to_print = [74, 150, 139, 80, 513, 71, 80, 437, 361, 499, 73, 74, 394, 507, 244, 547, 400, 463, 502, 270, 532, 418, 495, 64, 29]
rows_to_print = df.loc[indices_to_print]
print(rows_to_print)

                                       Chief Complaint Predict Consensus  \
74   R leg, R arm, bilateral hand pain and swelling...       U         Y   
150  pt c/o rt leg pain and swelling PMH: gout and HTN       U         Y   
139  left foot pain chronic but worse tonight. pmh ...       Y         Y   
80   L knee pain, "needs a new leg brace, and they ...       U         Y   
513  Rt. LE gout pain w/ swelling/redness to knee, ...       Y         Y   
71   dropped 10 lb weight on L foot, L great toe ap...       N         Y   
80   L knee pain, "needs a new leg brace, and they ...       U         Y   
437  pt c/o cough x2 weeks, OTC meds not helping. S...       U         Y   
361  gout flare up that started yesterday in left w...       Y         Y   
499  Right ankle pain, SOB, fluid retention, "irreg...       U         Y   
73           Gout flare in R knee x 1 day. Med hx: htn       Y         Y   
74   R leg, R arm, bilateral hand pain and swelling...       U         Y   
394  Left sh

In [23]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

X_biogpt_train, X_biogpt_test, y_biogpt_train, y_biogpt_test = train_test_split(X_biogpt, y_biogpt, test_size=0.3, random_state=42)
X_stan_train, X_stan_test, y_stan_train, y_stan_test = train_test_split(X_stan, y_stan, test_size=0.3, random_state=42)
X_llama_train, X_llama_test, y_llama_train, y_llama_test = train_test_split(X_llama, y_llama, test_size=0.3, random_state=42)

# Train the SVM models using the original training data (without oversampling)
svm_model_biogpt.fit(X_biogpt_train, y_biogpt_train)
svm_model_stanford.fit(X_stan_train, y_stan_train)
svm_model_llama.fit(X_llama_train, y_llama_train)

# Predict on the original test sets
y_biogpt_pred = svm_model_biogpt.predict(X_biogpt_test)
y_stan_pred = svm_model_stanford.predict(X_stan_test)
y_llama_pred = svm_model_llama.predict(X_llama_test)

# Print classification report for each model
print("Classification Report for BioGPT SVM Model lett without oversampling:")
print(classification_report(y_biogpt_test, y_biogpt_pred))

print("Classification Report for Stanford SVM Model lett without oversampling:")
print(classification_report(y_stan_test, y_stan_pred))

print("Classification Report for LLAMA SVM Model lett without oversampling:")
print(classification_report(y_llama_test, y_llama_pred))

Classification Report for BioGPT SVM Model lett without oversampling:
              precision    recall  f1-score   support

           N       0.97      0.92      0.94        74
           Y       0.25      0.50      0.33         4

    accuracy                           0.90        78
   macro avg       0.61      0.71      0.64        78
weighted avg       0.93      0.90      0.91        78

Classification Report for Stanford SVM Model lett without oversampling:
              precision    recall  f1-score   support

           N       0.97      0.89      0.93        74
           Y       0.20      0.50      0.29         4

    accuracy                           0.87        78
   macro avg       0.59      0.70      0.61        78
weighted avg       0.93      0.87      0.90        78

Classification Report for LLAMA SVM Model lett without oversampling:
              precision    recall  f1-score   support

           N       0.96      0.96      0.96        74
           Y       0.25   